# Arbol Final 1ra Competencia

### 01) Carga de librerias y Datos

In [1]:
# Arbol elemental con libreria  rpart
# Debe tener instaladas las librerias  data.table  ,  rpart  y  rpart.plot

# cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot") #Aqui se debe poner la ruta de la PC local

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot



In [2]:
# Aqui se debe poner la carpeta de la materia de SU computadora local (el Working Directory)
setwd("C:\\Users\\leandro.morinigo\\OneDrive - Accenture\\! DM en Econ y Fin 2023\\") 

#cargo el dataset
dataset <- fread("./datasets/competencia_01.csv") 

print(dim(dataset))

[1] 491063    155


In [3]:
# minima exploración por fecha
table(dataset$foto_mes)


202103 202104 202105 
163324 163637 164102 

In [4]:
# minima exploración por fecha y target
table(dataset$foto_mes, dataset$clase_ternaria )

        
                BAJA+1 BAJA+2 Continua
  202103      0   1049    963   161312
  202104   1363      0    963   161311
  202105   2790      0      0   161312

### 02) FEATURE ENGINEERING

In [5]:
# Corrige clientes con menos de 90 dias de antiguedad (al ser una medida trimestral)
#ctrx_quarter_fix
  dataset[  , ctrx_quarter_fix := ctrx_quarter ]
  dataset[ cliente_antiguedad==1 , ctrx_quarter_fix := ctrx_quarter * 3 ]
  dataset[ cliente_antiguedad==2 , ctrx_quarter_fix := ctrx_quarter * 2 ]

In [6]:
#Var del Top 20 que puedan servir en relacion a la edad y edad^2
# ctrx_quarter_fixed
  dataset[ , ctrx_quarter_fix_edad := ctrx_quarter_fix / cliente_edad ]
  dataset[ , ctrx_quarter_fix_edad2 := ctrx_quarter / cliente_edad^2]

# mprestamos_personales
  dataset[ ,mprestamos_personales_edad := mprestamos_personales  / cliente_edad ]
  dataset[ ,mprestamos_personales_edad2 := mprestamos_personales  / cliente_edad^2 ]

# mactivos_margen
  dataset[ ,mactivos_margen_edad := mactivos_margen  / cliente_edad ]
  dataset[ ,mactivos_margen_edad2 := mactivos_margen  / cliente_edad^2 ]

# mcomisiones
  dataset[ ,mcomisiones_edad := mcomisiones  / cliente_edad ]
  dataset[ ,mcomisiones_edad2 := mcomisiones  / cliente_edad^2 ]

# mpayroll
  dataset[ ,mpayroll_edad := mpayroll  / cliente_edad ]
  dataset[ ,mpayroll_edad2 := mpayroll  / cliente_edad^2 ]

In [7]:
# ACTIVIDAD: diferencia a la media al cuadrado
avg_ctrx_quarter_fix <- mean(dataset$ctrx_quarter_fix, na.rm = TRUE)
dataset[, ctrx_quarter_fix_square := (ctrx_quarter_fix - avg_ctrx_quarter_fix)^2]


#SALDO CUENTAS: Distancias
# mcuentas_saldo - mcaja_ahorro
dataset[, saldo_minus_caja := mcuentas_saldo - mcaja_ahorro]

# mcuentas_saldo - mcuenta_corriente
dataset[, saldo_minus_corriente := mcuentas_saldo - mcuenta_corriente]

# mcaja_ahorro - mcuenta_corriente
dataset[, caja_minus_corriente := mcaja_ahorro - mcuenta_corriente]

# mcuentas_saldo - mpayroll
dataset[, saldo_minus_payroll := mcuentas_saldo - mpayroll]



#PRESTAMOS: Ratios y Distancias
# mprestamos_personales / cprestamos_personales
dataset[, prestamos_ratio := ifelse(cprestamos_personales == 0, 100, mprestamos_personales / cprestamos_personales)]

# mprestamos_personales / mcuentas_saldo
dataset[, prestamos_saldo_ratio := ifelse(mcuentas_saldo <= 0, 100, mprestamos_personales / mcuentas_saldo)]

# mprestamos_personales - mcuentas_saldo
dataset[, prestamos_minus_saldo := mprestamos_personales - mcuentas_saldo]

# mprestamos_personales - max(0, mcuentas_saldo)
dataset[, prestamos_minus_nonnegative_saldo := mprestamos_personales - pmax(0, mcuentas_saldo)]

# mprestamos_personales / mpayroll
dataset[, prestamos_payroll_ratio := ifelse(mpayroll <= 0, 100, mprestamos_personales / mpayroll)]

# mprestamos_personales / cliente_antiguedad
dataset[, prestamos_antiguedad_ratio := ifelse(cliente_antiguedad <= 0, 100, mprestamos_personales / cliente_antiguedad)]



#RENTABILIDAD: Ratios y Distancias
# mrentabilidad - mcomisiones_mantenimiento
dataset[, rentabilidad_minus_comisiones := mrentabilidad - mcomisiones_mantenimiento]

# mrentabilidad / mcomisiones_mantenimiento
dataset[, rentabilidad_divided_by_comisiones := ifelse(mcomisiones_mantenimiento == 0, 100, mrentabilidad / mcomisiones_mantenimiento)]

# mrentabilidad - mactivos_margen
dataset[, rentabilidad_minus_activos := mrentabilidad - mactivos_margen]

# mrentabilidad / mactivos_margen
dataset[, rentabilidad_divided_by_activos := ifelse(mactivos_margen <= 0, 100, mrentabilidad / mactivos_margen)]

# mrentabilidad - mpasivos_margen
dataset[, rentabilidad_minus_pasivos := mrentabilidad - mpasivos_margen]

# mrentabilidad / mpasivos_margen
dataset[, rentabilidad_divided_by_pasivos := ifelse(mpasivos_margen <= 0, 100, mrentabilidad / mpasivos_margen)]

# mrentabilidad / ctrx_quarter_fix
dataset[, rentabilidad_divided_by_ctrx := ifelse(ctrx_quarter_fix == 0, 100, mrentabilidad / ctrx_quarter_fix)]

# mrentabilidad / (ctrx_quarter_fix - avg_ctrx_quarter_fix)^2
dataset[, rentabilidad_divided_by_square := ifelse(ctrx_quarter_fix_square == 0, 100, mrentabilidad / (ctrx_quarter_fix - avg_ctrx_quarter_fix)^2)]

# mcomisiones_mantenimiento / ctrx_quarter_fix
dataset[, comisiones_divided_by_ctrx := ifelse(ctrx_quarter_fix == 0, 100, mcomisiones_mantenimiento / ctrx_quarter_fix)]

# mcomisiones_mantenimiento / (ctrx_quarter_fix - avg_ctrx_quarter_fix)^2
dataset[, comisiones_divided_by_square := ifelse(ctrx_quarter_fix_square == 0, 100, mcomisiones_mantenimiento / (ctrx_quarter_fix - avg_ctrx_quarter_fix)^2)]

# mrentabilidad_annual / mrentabilidad
dataset[, rentabilidad_annual_divided_by_rentabilidad := ifelse(mrentabilidad == 0, 100, mrentabilidad_annual / mrentabilidad)]

# mrentabilidad_annual / mcomisiones_mantenimiento
dataset[, rentabilidad_annual_divided_by_comisiones := ifelse(mcomisiones_mantenimiento == 0, 100, mrentabilidad_annual / mcomisiones_mantenimiento)]


# TARJETAS:
# Cantidad de tarjetas total sum(ctarjeta_visa, ctarjeta_master)
dataset[, ctarjetas := rowSums(.SD, na.rm = TRUE), .SDcols = c("ctarjeta_visa", "ctarjeta_master")]

# sum(Visa_mlimitecompra, Master_mlimitecompra)
dataset[, suma_limites := Visa_mlimitecompra + Master_mlimitecompra]

# sum(Visa_msaldototal,Master_msaldototal)
dataset[, suma_saldos_total := Visa_msaldototal + Master_msaldototal]
# sum(Visa_msaldototal,Master_msaldototal) / mpayroll
dataset[, suma_saldos_total_divided_by_payroll := ifelse(mpayroll <= 0, 100, suma_saldos_total / mpayroll)]
# sum(Visa_msaldototal,Master_msaldototal) / mcuentas_saldo
dataset[, suma_saldos_total_divided_by_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_saldos_total / mcuentas_saldo)]
# sum(Visa_msaldototal,Master_msaldototal) / sum(Visa_mlimitecompra, Master_mlimitecompra)
dataset[, suma_saldos_total_divided_by_limites := ifelse(suma_limites == 0, 100, suma_saldos_total / suma_limites)]

# sum(Visa_msaldopesos,Master_msaldopesos)
dataset[, suma_saldos_pesos := Visa_msaldopesos + Master_msaldopesos]
# sum(Visa_msaldopesos,Master_msaldopesos) / mpayroll
dataset[, suma_saldos_pesos_divided_by_payroll := ifelse(mpayroll <= 0, 100, suma_saldos_pesos / mpayroll)]
# sum(Visa_msaldopesos,Master_msaldopesos) / mcuentas_saldo
dataset[, suma_saldos_pesos_divided_by_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_saldos_pesos / mcuentas_saldo)]
# sum(Visa_msaldopesos,Master_msaldopesos) / sum(Visa_mlimitecompra, Master_mlimitecompra)
dataset[, suma_saldos_pesos_divided_by_limites := ifelse(suma_limites == 0, 100, suma_saldos_pesos / suma_limites)]

# sum(Visa_mpagominimo,Master_mpagominimo)
dataset[, suma_pagos_minimos := Visa_mpagominimo + Master_mpagominimo]
# sum(Visa_mpagominimo,Master_mpagominimo) / mpayroll
dataset[, suma_pagos_minimos_divided_by_payroll := ifelse(mpayroll <= 0, 100, suma_pagos_minimos / mpayroll)]
# sum(Visa_mpagominimo,Master_mpagominimo) / mcuentas_saldo
dataset[, suma_pagos_minimos_divided_by_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_pagos_minimos / mcuentas_saldo)]
# sum(mtarjeta_visa_consumo, mtarjeta_master_consumo)
dataset[, suma_consumos := mtarjeta_visa_consumo + mtarjeta_master_consumo]

# sum(mtarjeta_visa_consumo, mtarjeta_master_consumo) / sum(ctarjeta_visa, ctarjeta_master)
dataset[, suma_consumos_divided_by_tarjetas := ifelse(ctarjetas == 0, 100, suma_consumos / ctarjetas)]
# sum(mtarjeta_visa_consumo, mtarjeta_master_consumo) / mpayroll
dataset[, suma_consumos_divided_by_payroll := ifelse(mpayroll <= 0, 100, suma_consumos / mpayroll)]
# sum(mtarjeta_visa_consumo, mtarjeta_master_consumo) / mcuentas_saldo
dataset[, suma_consumos_divided_by_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_consumos / mcuentas_saldo)]
# sum(mtarjeta_visa_consumo, mtarjeta_master_consumo) / sum(Visa_mlimitecompra, Master_mlimitecompra)
dataset[, suma_consumos_divided_by_limites := ifelse(suma_limites == 0, 100, suma_consumos / suma_limites)]

# sum(Visa_mpagado , Master_mpagado)
dataset[, suma_pagado := Visa_mpagado + Master_mpagado]
# sum(Visa_mpagado , Master_mpagado) - sum(Visa_msaldototal,Master_msaldototal)
dataset[, suma_pagado_minus_saldos := suma_pagado - suma_saldos_total]
# sum(Visa_mpagado , Master_mpagado) - sum(Visa_msaldopesos,Master_msaldopesos)
dataset[, suma_pagado_minus_saldos_pesos := suma_pagado - suma_saldos_pesos]
# sum(Visa_mpagado , Master_mpagado) - sum(mtarjeta_visa_consumo, mtarjeta_master_consumo)
dataset[, suma_pagado_minus_consumos := suma_pagado - suma_consumos]
# sum(Visa_mpagado , Master_mpagado) - mpayroll 
dataset[, suma_pagado_minus_consumos_pesos := suma_pagado - mpayroll ]

# RATIOS DE PAGO tarjetas
# Pago / SALDO TOTAL : sum(Visa_mpagado , Master_mpagado) / sum(Visa_msaldototal,Master_msaldototal)
dataset[, ratio_pagado_saldos := ifelse(suma_saldos_total == 0, 100, suma_pagado / suma_saldos_total)]
# Pago / SALDO $ : sum(Visa_mpagado , Master_mpagado) / sum(Visa_msaldopesos,Master_msaldopesos)
dataset[, ratio_pagado_saldos_pesos := ifelse(suma_saldos_pesos == 0, 100, suma_pagado / suma_saldos_pesos)]

# Pago / CONSUMO :sum(Visa_mpagado , Master_mpagado) / sum(mtarjeta_visa_consumo, mtarjeta_master_consumo)
dataset[, ratio_pagado_consumos := ifelse(suma_consumos == 0, 100, suma_pagado / suma_consumos)]
# Pago / Ingreso : sum(Visa_mpagado , Master_mpagado) / mpayroll
dataset[, ratio_pagado_payroll := ifelse(mpayroll <= 0, 100, suma_pagado / mpayroll)]
# sum(Visa_mpagado , Master_mpagado) / mcuentas_saldo
dataset[, ratio_pagado_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_pagado / mcuentas_saldo)]

# Deuda Total: sum(Visa_msaldototal,Master_msaldototal, mprestamos_personales)
dataset[, suma_saldos_prestamos := suma_saldos_total + mprestamos_personales]
# sum(Visa_msaldototal,Master_msaldototal, mprestamos_personales) / mpayroll
dataset[, suma_saldos_prestamos_divided_by_payroll := ifelse(mpayroll <= 0, 100, suma_saldos_prestamos / mpayroll)]
# sum(Visa_msaldototal,Master_msaldototal, mprestamos_personales) / mcuentas_saldo
dataset[, suma_saldos_prestamos_divided_by_saldo := ifelse(mcuentas_saldo <= 0, 100, suma_saldos_prestamos / mcuentas_saldo)]



In [8]:
  #FE2: Operaciones Varias --------------------------------------------------------------------------
  ## Tiempo de vida en el banco 
  dataset[, vida_banco := (cliente_antiguedad/12) / (cliente_edad)]
  
  ## Total Pasivos
  dataset[, total_deuda := rowSums(.SD, na.rm = TRUE), .SDcols = c("mprestamos_personales", "mprestamos_prendarios", "mprestamos_hipotecarios", "Visa_msaldototal", "Master_msaldototal")]
  
  ## Total Activos
  dataset[, total_activos := rowSums(.SD, na.rm = TRUE), .SDcols = c("mplazo_fijo_dolares", "mplazo_fijo_pesos", "minversion1_pesos", "minversion1_dolares", "minversion2", "mcuentas_saldo")]
  
  ## Tiene cuenta homebanking
  dataset[, has_internet := ifelse(dataset$internet > 0, 1, 0) ]
  
  ## Tiene movimientos/tarjetas
  dataset[, has_debito_transacciones := ifelse(dataset$ctarjeta_debito_transacciones > 0, 1, 0) ]
  dataset[, has_visa := ifelse(dataset$ctarjeta_visa > 0, 1, 0) ]
  dataset[, has_visa_transacciones := ifelse(dataset$ctarjeta_visa_transacciones > 0, 1, 0) ]
  dataset[, has_master := ifelse(dataset$ctarjeta_master > 0, 1, 0) ]
  dataset[, has_master_transacciones := ifelse(dataset$ctarjeta_master_transacciones > 0, 1, 0) ]
  
  ## Recibo pago de sueldo?
  dataset[, has_payroll := ifelse(dataset$cpayroll_trx + dataset$cpayroll2_trx  > 0, 1, 0) ]
    
  ## Tiene débitos automáticos?
  dataset[, has_da := ifelse(dataset$ccuenta_debitos_automaticos + dataset$ctarjeta_visa_debitos_automaticos + dataset$ctarjeta_master_debitos_automaticos  > 0, 1, 0) ]
  
  ## ¿hace más transacciones en cajeros de otros bancos?
  dataset[, cajeros_ajenos := ifelse(dataset$matm < dataset$matm_other, 1, 0)]

In [9]:
#Reglas arbol simple sin 2 variables vinculadas: -ctrx_quarter -active_quarter
dataset[  , z1 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales<12840&ctarjeta_master<0.5) ]
dataset[  , z2 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales<12840&ctarjeta_master>=0.5) ]
dataset[  , z3 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo<3215&mpasivos_margen<183&cpayroll_trx<0.5) ]
dataset[  , z4 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales<0.5&mautoservicio<699) ]

dataset[  , z5 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen<183&cpayroll_trx>=0.5&mcomisiones_mantenimiento>=1770) ]
dataset[  , z6 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales>=12840&cdescubierto_preacordado<0.5) ]
dataset[  , z7 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo<1840&mpasivos_margen>=183&cpayroll_trx<0.5) ]
dataset[  , z8 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo>=3215&mpasivos_margen<183&cpayroll_trx<0.5) ]

dataset[  , z9 :=  as.integer (mcaja_ahorro>=343&mpasivos_margen>=183&cpayroll_trx>=0.5) ]
dataset[  , z10 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo<1257&mprestamos_personales>=12840&cdescubierto_preacordado>=0.5) ]
dataset[  , z11 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales<0.5&mautoservicio>=699) ]
dataset[  , z12 :=  as.integer(mcaja_ahorro>=343&mtarjeta_visa_consumo>=1840&mpasivos_margen>=183&cpayroll_trx<0.5) ]


dataset[  , z13 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales>=0.5&Visa_mconsumospesos>=12699) ]
dataset[  , z14 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen<183&cpayroll_trx>=0.5&mcomisiones_mantenimiento<1770) ]
dataset[  , z15 :=  as.integer(mcaja_ahorro>=343&mpasivos_margen>=183&cpayroll_trx>=0.5&cliente_antiguedad>=3.5) ]
dataset[  , z16 :=  as.integer(mcaja_ahorro<343&mtarjeta_visa_consumo>=1257&cprestamos_personales>=0.5&Visa_mconsumospesos<12699) ]

In [10]:
head(dataset[,155:240],n=2)

clase_ternaria,ctrx_quarter_fix,ctrx_quarter_fix_edad,ctrx_quarter_fix_edad2,mprestamos_personales_edad,mprestamos_personales_edad2,mactivos_margen_edad,mactivos_margen_edad2,mcomisiones_edad,mcomisiones_edad2,⋯,has_da,cajeros_ajenos,z1,z2,z3,z4,z5,z6,z7,z8
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Continua,420,8.4,0.168,0,0,155.0212,3.100424,47.7964,0.955928,⋯,1,0,0,0,0,0,0,0,0,0
Continua,390,7.8,0.156,0,0,95.8244,1.916488,46.5312,0.930624,⋯,1,0,0,0,0,0,0,0,0,0


### 03) Data Drifting

In [ ]:
#Como seleccionar los campos monetarios
#campos_monetarios  <- colnames(dataset)
#campos_monetarios  <- campos_monetarios[campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)"]

In [ ]:
#Seleccion Manual de campos a corregir: 

cols_to_fix <- c("mcuentas_saldo", 
                  "mcuenta_corriente",
                  "mcomisiones", 
                  "Visa_mpagominimo",
                  "mcomisiones_otras",
                  "mcaja_ahorro_dolares",
                  "mtransferencias_recibidas")

In [ ]:
#FUNCION: FIX Drifting con "rank_cero_fijo"
    #los positivos se rankean por su lado
    #los negativos se rankean por su lado

drift_rank_cero_fijo  <- function( campos_drift )
{
  for( campo in campos_drift )
  {
    cat( campo, " " )
    dataset[ get(campo) ==0, paste0(campo,"_rank") := 0 ]
    dataset[ get(campo) > 0, paste0(campo,"_rank") :=   frank(  get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ get(campo) < 0, paste0(campo,"_rank") :=  -frank( -get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ , (campo) := NULL ]
  }
}

In [ ]:
#aplicar la funcion para corregir Data Drifting: NO LA APLIQUE
#La prueba con solo DataDrifting al dataset original empeoro el resultado del score publico -> por eso NO LA APLICO en este caso
#pero cuando vi luego el score privado, habia Mejorado -> probar incluila en el caso completo y ver si mejora 

#drift_rank_cero_fijo( cols_to_fix )

### 02) Modelado Rpart

In [11]:
#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes==202103, 
         clase_binaria :=  ifelse( clase_ternaria=="Continua", "0L", "1L" ) ]

In [12]:
# SEPARO EN TRAIN(dtrain) Y TEST(dapply):
dtrain <- dataset[foto_mes == 202103] # defino donde voy a entrenar
dapply <- dataset[foto_mes == 202105] # defino donde voy a aplicar el model

print(paste("train:", dim(dtrain)[1], "| test:", dim(dapply)[1]))

[1] "train: 163324 | test: 164102"


**HIPERPARAMETROS** DEL MODELO

In [13]:
# genero el modelo,  aqui se construye el arbol
# quiero predecir clase_ternaria a partir de el resto de las variables
modelo  <- rpart(formula=   "clase_binaria ~ . -clase_ternaria",
                 data=      dtrain,  #los datos donde voy a entrenar
                 xval=         0,
                 cp=          -1,   # esto significa no limitar la complejidad de los splits
                 minsplit=  1119,    # minima cantidad de registros para que se haga el split  
                 minbucket=  245,   # tamaño minimo de una hoja  
                 maxdepth=     14 ) # profundidad maxima del arbol

### 03) Prediccion

In [14]:
# aplico el modelo a los datos nuevos
prediccion <- predict(
        object = modelo,
        newdata = dapply,
        type = "prob"
)

In [15]:
# prediccion es una matriz con DOS columnas,
# cada columna es el vector de probabilidades
head(prediccion, n=2)

,0L,1L
1,0.9998768,0.0001232267
2,1.0000000,0.0000000000


In [16]:
# la ordeno de mayor a menor prob de baja+2
prediccion_df <- as.data.frame(prediccion)

head(prediccion_df[order(-prediccion_df$`1L`), ], n=3)

,0L,1L
,<dbl>,<dbl>
161,0.6505051,0.3494949
268,0.6505051,0.3494949
395,0.6505051,0.3494949


In [17]:
# creo un df dfinal, que solo tiene el numero de cliente y la probabilidad para BAJA+2 = 1L
dfinal  <- copy( dapply[ , list(numero_de_cliente) ] )
dfinal[ , prob_SI := prediccion[ , "1L"] ]

head(dfinal ,n=3)

numero_de_cliente,prob_SI
<int>,<dbl>
29183981,0.0001232267
29184630,0.0000000000
29185433,0.0000000000


### 04) Cortes para Envios

In [18]:
# solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40
set.seed(130631)  
dfinal[ , azar := runif( nrow(dapply) ) ]

# ordeno en forma descentente, y cuando coincide la probabilidad, al azar
setorder( dfinal, -prob_SI, azar )


head(dfinal ,n=7)

numero_de_cliente,prob_SI,azar
<int>,<dbl>,<dbl>
124028204,0.3494949,0.0004523343
104907405,0.3494949,0.0016011221
39484959,0.3494949,0.0021200685
68497913,0.3494949,0.0045184768
104555124,0.3494949,0.0070446474
81273109,0.3494949,0.0076577596
102927621,0.3494949,0.0103584635


### 05) Exporto el archivo con las predicciones

In [19]:
# primero creo la carpeta donde va el experimento
dir.create("./exp/")
dir.create("./exp/KA4120")

Warning message in dir.create("./exp/"):
"'.\exp' already exists"
Warning message in dir.create("./exp/KA4120"):
"'.\exp\KA4120' already exists"


In [20]:
# genero el archivo para Kaggle: EL MEJOR RESULTADO SE OBTUVO CON UN CORTE DE 7500 ENVIOS
for( corte  in  c( 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000 ) )
{
  #le envio a los  corte  mejores,  de mayor probabilidad de prob_SI
  dfinal[ , Predicted := 0L ]
  dfinal[ 1:corte , Predicted := 1L ]


  fwrite( dfinal[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
           file= paste0( "./exp/KA4120/08_BOFinal_KA4120_005_",  corte, ".csv"),
           sep=  "," )
}

### 06) Calcular el feature importance


In [21]:
# Calcular el feature importance
importancia <- modelo$variable.importance
importancia <- data.frame(Variable = names(importancia), Importancia = importancia)

head(importancia, n=40)

,Variable,Importancia
,<chr>,<dbl>
z1,z1,165.965663
z2,z2,70.736319
prestamos_minus_saldo,prestamos_minus_saldo,41.368578
rentabilidad_annual_divided_by_comisiones,rentabilidad_annual_divided_by_comisiones,36.461397
mcuentas_saldo,mcuentas_saldo,33.999829
total_activos,total_activos,33.776206
z3,z3,30.701569
saldo_minus_caja,saldo_minus_caja,25.906626
saldo_minus_payroll,saldo_minus_payroll,25.348009


In [22]:
fwrite(importancia,
        file = "./exp/KA4120/08_BOFinal_KA4120_005_FeImportance.csv",
        sep = ",")